## NSynth Datasetのデータフォーマット確認

https://magenta.tensorflow.org/datasets/nsynth#format

まずはセットアップ

In [0]:
#@title Set Sound Length (in Seconds) { vertical-output: true, run: "auto" }
sample_time = 4.0 #@param {type:"number"}
sr = 16000
length = int(sr * sample_time)
# gansynthのデータは (length/sr) = 4sec が使われている。実装確認済み
# https://github.com/tensorflow/magenta/blob/master/magenta/models/gansynth/lib/datasets.py#L83

In [0]:
# NSynthの小さめのテストデータをダウンロード
!wget http://download.magenta.tensorflow.org/datasets/nsynth/nsynth-test.tfrecord

In [0]:
import tensorflow as tf

filename = 'nsynth-test.tfrecord'
filename_queue = tf.train.string_input_producer([filename])
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

example = tf.parse_single_example(
    serialized_example,
      features = {
          'pitch': tf.FixedLenFeature([1], dtype=tf.int64),
          'audio': tf.FixedLenFeature([length], dtype=tf.float32),
          'qualities': tf.FixedLenFeature([10], dtype=tf.int64),
          'instrument_source': tf.FixedLenFeature([1], dtype=tf.int64),
          'instrument_family': tf.FixedLenFeature([1], dtype=tf.int64),
      })


In [0]:
# データの取り出しと保存
out_filename = 'out_file.tfrecord'
with tf.python_io.TFRecordWriter( out_filename ) as writer:
  for record in tf.python_io.tf_record_iterator(filename):
    example = tf.train.Example()
    example.ParseFromString(record)
    # gansynth では次の5つの値を使っている。audio以外は訓練データに使わないように除外するフィルタとして利用している
    # https://github.com/tensorflow/magenta/blob/master/magenta/models/gansynth/lib/datasets.py#L114
    p = example.features.feature["pitch"].int64_list.value[0] # この音階のみ4secに入っている。細かい説明はNSynth Datasetの冒頭にあり
    a = example.features.feature["audio"].float_list.value # 実データ 64000 length
    q = example.features.feature["qualities"].int64_list.value
    inst_src = example.features.feature["instrument_source"].int64_list.value[0]
    inst_fam = example.features.feature["instrument_family"].int64_list.value[0]

    if inst_src == 0: # accoustic only 
      w_example = tf.train.Example(features=tf.train.Features(feature={
                          "pitch": tf.train.Feature(int64_list=tf.train.Int64List(value=[p])),
                          "audio": tf.train.Feature(float_list=tf.train.FloatList(value=a)),
                          "qualities": tf.train.Feature(int64_list=tf.train.Int64List(value=q)),
                          "instrument_source": tf.train.Feature(int64_list=tf.train.Int64List(value=[inst_src])),
                          "instrument_family": tf.train.Feature(int64_list=tf.train.Int64List(value=[inst_fam]))
                          }))

      writer.write(w_example.SerializeToString())

## 保存したデータの確認

参考サイト: 2018/5/19にPyCon mini Osakaで「librosaで始める音楽情報検索」
http://www.hiromasa.info/posts/5/

In [0]:
# データの取り出し

import IPython.display

NUM_THREADS = 8
BATCH_SIZE = 100

def _parse_function(serialized):
    features={
          'pitch': tf.FixedLenFeature([1], dtype=tf.int64),
          'audio': tf.FixedLenFeature([length], dtype=tf.float32),
          'qualities': tf.FixedLenFeature([10], dtype=tf.int64),
          'instrument_source': tf.FixedLenFeature([1], dtype=tf.int64),
          'instrument_family': tf.FixedLenFeature([1], dtype=tf.int64),
        }
    p = tf.parse_single_example(serialized, features)  # データ構造を解析
    
    return (p["pitch"], 
            p["audio"], 
            p["qualities"], 
            p["instrument_source"], 
            p["instrument_family"])
  
with tf.Session() as sess:
  dataset = tf.data.TFRecordDataset(out_filename).shuffle(True).map(_parse_function, NUM_THREADS).batch(BATCH_SIZE)

  iter = dataset.make_one_shot_iterator()  # イテレータを初期化
  item = iter.get_next()  # イテレータの次の要素を取得
  sess.run(tf.global_variables_initializer())

  while(True):
    (p,a,q,inst_src,inst_fam) = sess.run(item)
    #example = tf.train.Example()
    #example.ParseFromString(record)
    print('{}'.format(a[0]))
    break

In [0]:
IPython.display.Audio(data=a[10], rate=sr)

In [0]:
import librosa.display
import numpy as np

# 波形の描画
librosa.display.waveplot(np.asarray(a[10]), sr)

In [0]:
import matplotlib.pyplot as plt

# 短時間フーリエ変換
D = librosa.stft(a[10])

# スペクトグラム
fig = plt.figure(1, figsize=(12,4)); ax = fig.add_subplot(1,1,1)

log_power = librosa.amplitude_to_db(np.abs(D), ref=np.max)
librosa.display.specshow(log_power, x_axis="time", y_axis="linear")
plt.colorbar()

### 新たにデータセットを作れるのか？

作ってみようかと思ったが、1レコードを1音階に制限して作ることがわかり、市販の音源を1音階のみ取り出すのは容易ではないことが想定され、オリジナルまたは別のデータセットをどのように揃えるか課題がある。

In [0]:
import librosa

sr = 16000
sample_length = 64000
audio, _ = librosa.load('generated_clip.wav', sr=sr)
audio = audio[:sample_length]
len(audio)